# Homework 8

Author: Mao Nishino

In [1]:
# Import relevant libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from IPython.display import display_markdown

## Problem 1
Download the dataset hmm_pb1.csv from Canvas. It represents a sequence of
dice rolls x from the Dishonest casino model discussed in class. The model parameters
are exactly those presented in class. The states of Y are 1=’Fair’ and 2=’Loaded’.

In [2]:
# Loads the sequence of rolls. 
# 1D array where each element represents the result of each dice roll.
pb1 = np.array(pd.read_csv('./hmm_pb1.csv', header=None)).squeeze()

In [3]:
# Define the HMM
"""
Data Structures
    pi(np.ndarray): The initial probabilities. 1D array of size 2.
                    pi[0] = P(the first dice is fair)
                    pi[1] = P(the first dice is loaded)

    a(np.ndarray): The transition probabilities. 2d array of size (2,2).
                    a[i,j] = P(Y_{n+1}=j+1|Y_n = i+1)

    b(np.ndarray): The emission probabilities. 2d array of size (2,6).
                   b[i,j] = P(X_n=j+1|Y_n=i+1)
    Here, Y_n = the hidden state. Y_n=1=>fair, Y_n=2=>loaded.
    X_n = the observed dice roll at time n.    
"""
pi = np.array([0.5, 0.5])
a = np.array(
    [[0.95, 0.05],
     [0.05, 0.95]]
)
b = np.array(
    [[1./6, 1./6, 1./6, 1./6, 1./6, 1./6],
    [1./10, 1./10, 1./10, 1./10, 1./10, 1./2]]
)

### Problem (a)
Implement the Viterbi algorithm and find the most likely sequence y that gener-
ated the observed x. Use the log probabilities, as shown in the HMM slides from
Canvas. Report the obtained sequence y of 1’s and 2’s for verification. (2 points)

In [4]:
"""
Data Structures Specific To This Problem
    c (np.ndarray): the log Viterbi probability. 
        It has 2 rows and pb1.shape[0] columns.
        The kth row represents the log-probabilities at each time for
        the hidden state k+1. (k=0=>Fair, k=1=>loaded)
        The jth column represents the log-probabilities for each hidden state
        at time j.

    ptr (np.ndarray): the hidden states that maximizes
                       the Viterbi probability at each time.
                       The format is the same as -C.
                       Ptr in the page 27 of HMM slides.

    y_star (np.ndarray): the most likely sequence of the hidden state.
"""

c = np.zeros((2, pb1.shape[0]))
ptr = np.zeros((2, pb1.shape[0]))
y_star = -1*np.ones_like(pb1) # Put -1 to make debugging easier

"""
The Viterbi Algorithm
"""

# Initialization
c[:,0] = np.log(b[:, pb1[0]-1]*pi)

# Loops
for t in range(1, pb1.shape[0]):
    # Reshape so that c[:,t-1] would be added to each column of log(a) 
    # by broadcasting
    logaC = np.log(a) + c[:, t-1].reshape(-1,1)
    c[:, t] = np.log(b[:, pb1[t]-1]) + np.max(logaC, axis = 0)
    ptr[:, t] = np.argmax(logaC, axis = 0)

# We now find y_star.
# Initialization
y_star[-1] = np.argmax(c[:, -1])

# Start from the last index pb1.shape[0]-1
# last -1 means we go backward
for t in range(pb1.shape[0]-1, 0, -1):
    y_star[t-1] = ptr[y_star[t], t]

display_markdown("The following shows the most probable sequence of the hidden state.", raw = True)
print(y_star+1)

The following shows the most probable sequence of the hidden state.

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]


### Question (b)
Implement the forward and backward algorithms and run them on the observed
x. You should memorize a common factor ut for the αkt to avoid floating point
underflow, since αkt quickly become very small. The same holds for $β_{t}^{k}$ . Report
$\alpha_{135}^1/\alpha_{135}^2/$ and $\beta_{135}^1/\beta_{135}^2$, where the counting starts from t = 0. (3 points)

In [9]:
"""
Data structures specific to this problem

alpha (np.ndarray) : the forward probability of the shape 2 x pb.shape[0]
        The kth row represents the forward-probabilities at each time for
        the hidden state k+1. (k=0=>Fair, k=1=>loaded)
        The jth column represents the backward probabilities for each hidden state
        at time j.
beta (np.ndarray): the backward probability of the shape 2 x pb.shape[0]
"""

def forward_backward(pb: np.ndarray, pi, a, b):
    """ The Forward Backward Algorithm

    Args:
    pb: the dataset
    pi: initial prob
    a: transition prob
    b: emission prob
    
    Returns:
    alpha: the forward probability
    beta: the backward probability
    """

    alpha = np.zeros((2, pb.shape[0]))
    beta = np.zeros((2, pb.shape[0]))

    # Find the foward probability
    alpha[:, 0] = b[:, pb[0]-1]*pi

    for t in range(1, pb.shape[0]):
        alpha_numerator = b[:, pb[t]-1] * np.sum(a*(alpha[:,t-1].reshape(-1,1)), axis = 0)
        # Normalize to avoid underflow
        alpha_denominator = np.sum(alpha_numerator)
        alpha[:, t] = alpha_numerator/alpha_denominator

    # Find the backward probability
    beta[:, -1]  = 1

    for t in range(pb.shape[0]-2, -1, -1):
        beta_numerator = np.sum(a*(beta[:,t+1]*b[:, pb[t+1]-1]), axis = 1)
        # Normalize to avoid underflow
        beta_denominator =  np.sum(beta_numerator)
        beta[:, t] = beta_numerator/beta_denominator

    return alpha, beta

# Display the result
alpha, beta = forward_backward(pb1, pi, a, b)
display_markdown("The following shows $\\alpha_{135}^1/\\alpha_{135}^2$.", raw=True)
print([alpha[0][t]/alpha[1][t] for t in [1,3,5]])
display_markdown("The following shows $\\beta_{135}^1/ \\beta_{135}^2$.", raw=True)
print([beta[0][t]/beta[1][t] for t in [1,3,5]])

The following shows $\alpha_{135}^1/\alpha_{135}^2$.

[2.6344086021505375, 5.5035735614248, 8.748795642368156]


The following shows $\beta_{135}^1/ \beta_{135}^2$.

[4.201606143659003, 2.2675804409135245, 0.9492586264514562]


# Problem 2
Download the dataset hmm_pb2.csv from Canvas. It represents a sequence of
10000 dice rolls x from the Dishonest casino model but with other values for the a and
b parameters than those from class. Having so many observations, you are going to
learn the model parameters.
Implement and run the Baum-Welch algorithm using the forward and backward
algorithms that you already implemented for Pb 1. You can initialize the π,a,b with
your guess, or with some random probabilities (make sure that π sums to 1 and that
aij ,bik sum to 1 for each i). The algorithm converges quite slowly, so you might need
to run it for up 1000 iterations or more for the parameters to converge.
Report the values of π,a,b that you have obtained. (4 points)
1

In [6]:
# Loads the sequence of rolls. 
# 1D array where each element represents the result of each dice roll.
pb2 = np.array(pd.read_csv('./hmm_pb2.csv', header=None)).squeeze()

In [8]:
def baum_welch(pb, pi, a, b, niter = 10):
    """ Baum-Welch Algorithm
    
    Given the dataset, it learns a HMM using the Baum-Welch algorithm.

    Args:
        pb: the dataset.
        pi: the initial guess on the initial probablity.
        a: the initial guess on the transition probablity.
        b: the initial guess on the emission probablity.
    Returns:
        _pi: the initial probabilty.
        _a: the transition probability.
        _b: the emission probability.
        Formats are the same as previously defined.
    """

    # Initialization
    _pi = pi
    _a = a
    _b = b

    # Create delta(x_t = k).
    # It should have the shape 6*pb.shape[0].
    encoder = OneHotEncoder(categories = [[1,2,3,4,5,6]], sparse_output=False)
    encoded_pb = np.transpose(encoder.fit_transform(pb.reshape(-1,1)))
    
    for i in range(niter):
        alpha, beta = forward_backward(pb, _pi, _a, _b)
        # E-Step

        # Calculation of xi. xi should have the size 2 x 2 x pb.shape[0],
        # where the first index represents i, the second j, the last t
        # where i,j,t are indexes from the slides.
        
        # Calculate b_x(t+1)^j
        bx = _b[:, np.roll(pb-1,-1)]

        xi_numerator = alpha.reshape(2, 1, -1) * \
            _a.reshape(2, 2, 1) * \
            np.roll(beta.reshape(1, 2, -1), shift = -1, axis = -1) * \
            bx.reshape(1, 2, -1)
        
        xi_denominator = np.sum(xi_numerator, axis=(0,1)).reshape(1,1,-1)
        xi = xi_numerator/xi_denominator
        assert xi_denominator.min() != 0

        # Calculation of gamma. gamma should have size 2 x pb.shape[0].
        gamma_numerator = alpha*beta
        gamma_denominator = np.sum(gamma_numerator, axis = 0).reshape(1,-1)
        gamma = gamma_numerator/gamma_denominator
        assert gamma_denominator.min() != 0

        # M-step
        _pi = gamma[:, 0]
        _a = np.sum(xi[:,:,:-1], axis = 2)/np.sum(gamma[:, :-1], axis = 1).reshape(-1,1)
        _b = np.sum(gamma.reshape(2,1,-1)*encoded_pb.reshape(1,6,-1), axis = 2)/ \
            np.sum(gamma, axis = 1).reshape(-1,1)
    
    return _pi, _a, _b

pi = np.array([0.5, 0.5])
a = np.array(
    [[0.95, 0.05],
     [0.05, 0.95]]
)
b = np.array(
    [[1./6, 1./6, 1./6, 1./6, 1./6, 1./6],
    [1./10, 1./10, 1./10, 1./10, 1./10, 1./2]]
)

new_pi, new_a, new_b = baum_welch(pb2, pi, a, b, 1000)
display_markdown("The following shows the learned initial probability.", raw=True)
print(new_pi)
display_markdown("The following shows the learned transition probability.", raw=True)
print(new_a)
display_markdown("The following shows the learned emission probability.", raw=True)
print(new_b)

The following shows the learned initial probability.

[4.90171704e-294 1.00000000e+000]


The following shows the learned transition probability.

[[0.70708794 0.29291206]
 [0.01238918 0.98761082]]


The following shows the learned emission probability.

[[0.09569917 0.11414408 0.07285264 0.04437644 0.57558343 0.09734424]
 [0.20097134 0.20571544 0.19349497 0.20178681 0.10500669 0.09302474]]
